In [1]:
using Random
using JuMP
using Gurobi    
using Distributions
using DataFrames
using CSV
using Dates
using Base.Filesystem

using DataStructures

In [2]:
include("instance_generation.jl")
include("RO_functions.jl")

fullRO_master_decomposition_NN_adv_NN (generic function with 1 method)

# Two Stage Knapsack RO problem

The approach is to fix the first stage decisions, this is done by solving a MIP in x by completly ignoring the uncertainities and y               
Then Dirchlet distribution is used to generate budget uncertainities for each profit              
The orginal problem reduces to a another MIP in y, where x and uncertainities are fixed: The objective of the this problem represents                  
the vakue of NN model with  x, problem instance values and uncertainties are the input

In [4]:
function solve_knapsack(model, weights, profits, capacity; verbose=false)
    n = length(weights)
    # Set the verbose parameter
    
    
    # Create binary decision variables
    @variable(model, x[1:n], Bin)
    
    # Set objective function
    @objective(model, Max, sum(profits[i] * x[i] for i in 1:n))
    
    # Add constraint: total weight should not exceed capacity
    @constraint(model, sum(weights[i] * x[i] for i in 1:n) <= capacity)
    
    # Solve the model
    optimize!(model)
    
    # Get the optimal solution
    solution = Dict()
    solution["objective_value"] = objective_value(model)
    solution["selected_items"] = value.(x) #[i for i in 1:n if value(x[i]) > 0.5]
    
    return solution
end

solve_knapsack (generic function with 1 method)

In [5]:
"""
    second_stage(X, info)

The `second_stage` function solves a second-stage optimization problem for a knapsack problem.

# Arguments
- `X`: A vector of binary decision variables representing the first-stage solution.
- `info`: A dictionary containing the problem information including the following keys:
    - `"f"`: A vector of first-stage objective function coefficients.
    - `"p_bar"`: A vector of first-stage constraint coefficients.
    - `"p_hat"`: A vector of second-stage constraint coefficients.
    - `"t"`: A vector of second-stage constraint coefficients.
    - `"C"`: The capacity constraint.
    - `"Xi"`: A vector of uncertainty values.
    - `"w"`: A vector of weights.

# Returns
- `solution`: A dictionary containing the optimal solution with the following keys:
    - `"objective_value"`: The objective value of the second-stage problem.
    - `"y"`: A vector of binary decision variables representing the second-stage solution.
    - `"r"`: A vector of binary decision variables representing an intermediate variable.
    - `"x"`: A vector of binary decision variables representing the first-stage solution.

"""
function second_stage(model, X, p_bar, p_hat, t, f, C, uncern, w, verbose=false)

    
    n = length(X)
    
    # Create binary decision variables
    #@variable(model, x[1:n], Bin) 
    @variable(model, y[1:n], Bin)
    @variable(model, r[1:n], Bin) 
    

    # Set objective function

    @objective(model, Min, (sum((p_hat[i] * uncern[i] - f[i]) * y[i] - p_hat[i] * uncern[i] * r[i] for i in 1:n)))

    # Add constraint: Select y if x is selected, 
    ## Fix to the first stage solution, the selected X is only used to maintain feasibility for the second stage
    # @constraint(model, [i in 1:n], x[i] == X[i])

    # Add constraint: Select y if x is selected
    @constraint(model, [i in 1:n], y[i] <= X[i])

    # Add constraint: select r if y is selected
    @constraint(model, [i in 1:n], r[i] <= y[i])

    # Add constraint: total weight should not exceed capacity
    @constraint(model, sum(t[i] * r[i] + w[i] * y[i] for i in 1:n) <= C)
    
    # Solve the model

    optimize!(model)

    # Get the optimal solution
    solution = Dict()   
    solution["ss_objective_value"] = objective_value(model)
    solution["y"] = value.(y) #[i for i in 1:n if value(y[i]) > 0.5]
    solution["r"] = value.(r) #[i for i in 1:n if value(r[i]) > 0.5]
    solution["x"] = value.(X) #[i for i in 1:n if value(r[i]) > 0.5]
    solution["fs_objective_value"] = sum((f[i] - p_bar[i]) * X[i] for i in 1:n)   #### add to the solution the first stage objective value
    solution["total_objective_value"] = solution["fs_objective_value"] + solution["ss_objective_value"] 
    return solution
end

second_stage

In [6]:
info_1=knapsack_instances(1,2,3)

Dict{String, Any} with 12 entries:
  "f"                             => [3.18513, 2.89089]
  "C"                             => 2.25364
  "h"                             => 80
  "delta"                         => 0.1
  "p_bar"                         => [2.39765, 2.25653]
  "t"                             => [1.09835, 1.11714]
  "p_hat"                         => [1.26361, 1.19156]
  "w"                             => [1.14673, 1.69848]
  "Matrix_of_Xi"                  => [0.008 0.182; 0.037 0.153; … ; 0.1873 0.00…
  "profit_uplim"                  => [1.31871, 1.24109]
  "budget_uncertainity_parameter" => 0.2
  "profit_downlim"                => [1.07894, 1.01544]

In [7]:
info_1["Matrix_of_Xi"][1,:]

2-element Vector{Float64}:
 0.008
 0.182

In [7]:
info_1["p_bar"]


2-element Vector{Float64}:
 2.397653367382937
 2.2565294806850034

## Combining the functions together

### Create a function that creates an instance of the problem internally by taking a seed

The Created instance has the opprtunity to reduce capacity of the Knapsack such that the first stage deterministic program results are suboptimal

Create L instances of the problem, for each instance create M optimal to suboptimal first stage
For each first stage decision, create N scenarios and solve the second stage MIP

Thus in total L x M first stage MIP and L x M x N MIP

Outputs L x M x N results


In [8]:
function ss_per_instances(fs_model_env, ss_model_env, folder_location, seed, num_items, num_scenarios, capacity_reduction_list=[0.75, 0.775, 0.8,  0.825, 0.85, 0.875, 0.90, 0.925, 0.95, 0.975, 1], instance::Function=knapsack_instances, solve_knapsack::Function=solve_knapsack)
    # Check if the folder exists, if not, create it
    if !isdir(folder_location)
        mkdir(folder_location)
    end
    # Generate random data for the knapsack problem
    info = knapsack_instances(seed, num_items; train=true, num_scenarios=num_scenarios)
    # Initialize an empty list to store output dictionaries
    output_list = []
    p_bar = info["p_bar"]
    p_hat=info["p_hat"]
    t = info["t"]
    f = info["f"]
    C = info["C"]
    uncern = info["Matrix_of_Xi"]
    w = info["w"]
    gamma = info["budget_uncertainity_parameter"]
    # Create a model for the first stage problem
    
    # Iterate over the capacity reduction factors
    for reduction in capacity_reduction_list
        # Solve the first stage knapsack problem
        fs_model = Model(optimizer_with_attributes(() -> Gurobi.Optimizer(fs_model_env)))
        set_optimizer_attribute(fs_model, "OutputFlag", 0)
        solution = solve_knapsack(fs_model, w, p_bar, C * reduction)

        for i in 1:num_scenarios
            X = solution["selected_items"]
            # Solve the second stage problem
            ss_model = Model(optimizer_with_attributes(() -> Gurobi.Optimizer(ss_model_env)))
            set_optimizer_attribute(ss_model, "OutputFlag", 0)
            second_stage_solution = second_stage(ss_model, X, p_bar, p_hat, t, f, C, uncern[i,:], w)

            r=second_stage_solution["r"]
            y=second_stage_solution["y"]
            x=second_stage_solution["x"]
            ss_obj=second_stage_solution["ss_objective_value"]
            fs_obj=second_stage_solution["fs_objective_value"]
            total_obj = second_stage_solution["total_objective_value"]
            #println("Second stage objective value: ", second_stage_solution["objective_value"])
            #println("Second stage solution: ", second_stage_solution)

            output = Dict("p_bar" => p_bar, "p_hat" => p_hat, "t" => t, "f" => f, "uncern" => uncern[i,:], "w" => w, "first_stage_obj" => fs_obj, "r"=>r, "y" => y, "x" => x, "second_stage_obj" => ss_obj, "total_obj"=> total_obj, "Reduced Capacity" => C * reduction, "original_capacity" => C, "gamma" => gamma, "seed" => seed)
            #println(output)
            # Append the output dictionary to the output list
            push!(output_list, output)
        end

    end

    df = DataFrame(output_list)
    
    #Generate a timestamp for the filename
    timestamp = Dates.format(now(), "yyyy-mm-dd_HH-MM-SS")
    
    #Construct the filename with the timestamp and seed
    filename = "output_$seed-$timestamp.csv"
    filename = joinpath(folder_location, filename)
    #Save the DataFrame to CSV
    CSV.write(filename, df)

    return 0
end

ss_per_instances (generic function with 4 methods)

In [9]:
parent_dir = "c:\\Users\\dube.rohit\\OneDrive - Texas A&M University\\ROoptjulia"
folder_path = "pre train NN data"

lower_instance = 1       ##### lower seed
list_to_create_x = [0.75, 0.775, 0.8,  0.825, 0.85, 0.875, 0.90, 0.925, 0.95, 0.975, 1]     #### Capacity reduction list
num_first_stage = length(list_to_create_x)
upper_instance = 250      ##### upper seed
num_items = 20
num_scenarios = 50        ##### Number of scenarios

# Create the folder
fs_model_env = Gurobi.Env()
ss_model_env = Gurobi.Env()
folder_name = "instance_$(lower_instance)_$(upper_instance)_items_$(num_items)_num_of_first_stage_$(num_first_stage)_scenarios_$(num_scenarios)_total_obj"
full_filepath = joinpath(parent_dir, folder_path, folder_name)

for instance in lower_instance:upper_instance
    ss_per_instances(fs_model_env, ss_model_env, full_filepath, instance, num_items, num_scenarios, list_to_create_x)    ### seed = instance
end

Set parameter Username
Academic license - for non-commercial use only - expires 2025-02-14
Set parameter Username
Academic license - for non-commercial use only - expires 2025-02-14


## Function to solve the problem with one uncertainty

In [4]:
knapsack_instances(1,10; train=true, num_scenarios=1000)

Dict{String, Any} with 12 entries:
  "f"                             => [1281.67, 684.006, 1171.88, 796.505, 259.6…
  "C"                             => 1765.35
  "h"                             => 40
  "delta"                         => 1.0
  "p_bar"                         => [944.374, 461.589, 830.504, 573.559, 177.4…
  "t"                             => [43.0911, 11.9223, 89.2382, 21.1985, 117.5…
  "p_hat"                         => [599.11, 142.412, 57.0106, 533.624, 28.171…
  "w"                             => [50.1227, 119.96, 393.878, 25.0702, 692.16…
  "Matrix_of_Xi"                  => [0.0059 0.1341 … 0.0098 0.0159; 0.096 0.25…
  "profit_uplim"                  => [944.374, 461.589, 830.504, 573.559, 177.4…
  "budget_uncertainity_parameter" => 1.0
  "profit_downlim"                => [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0…

In [5]:
I=30
output_list = []
model_exact_env = Gurobi.Env()
for seed in 1:250
    instance = knapsack_instances(seed,I; train=true, num_scenarios=1000)
    f = instance["f"]
    p_bar = instance["p_bar"]
    p_hat = instance["p_hat"]
    t = instance["t"]
    C = instance["C"]
    uncern = instance["Matrix_of_Xi"]
    w = instance["w"]
    gamma = instance["budget_uncertainity_parameter"]
    uncertainty_matrix = instance["Matrix_of_Xi"]

    uncertainty_dict = OrderedDict()

    num_uncerntainties = size(uncertainty_matrix)[1]
    println("Solving Seed: ", seed)
    for i in 1:num_uncerntainties
        uncertainty_dict[1] = uncertainty_matrix[i,:]
        result_per_uncertainty = master_stage_exact(model_exact_env, I, uncertainty_dict, f, p_bar, t, p_hat, C, w)
        X = result_per_uncertainty["X"]
        #println("objective value: ", result_per_uncertainty["objective_value"])
        results = Dict("objective_value" => result_per_uncertainty["objective_value"], "f"=>f, "p_bar"=>p_bar, "p_hat"=>p_hat, "t"=>t, "C"=>C, "w"=>w, "uncertainty"=>uncertainty_dict[1], "gamma"=>gamma, "seed"=>seed, "X"=>X)
        push!(output_list, results)
    end
end

df = DataFrame(output_list)
file_name = "output_exact_pretrain_I=$(I).csv"
CSV.write(file_name, df)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-02-14
Solving Seed: 1
Solving Seed: 2
Solving Seed: 3
Solving Seed: 4
Solving Seed: 5
Solving Seed: 6
Solving Seed: 7
Solving Seed: 8
Solving Seed: 9
Solving Seed: 10
Solving Seed: 11
Solving Seed: 12
Solving Seed: 13
Solving Seed: 14
Solving Seed: 15
Solving Seed: 16
Solving Seed: 17
Solving Seed: 18
Solving Seed: 19
Solving Seed: 20
Solving Seed: 21
Solving Seed: 22
Solving Seed: 23
Solving Seed: 24
Solving Seed: 25
Solving Seed: 26
Solving Seed: 27
Solving Seed: 28
Solving Seed: 29
Solving Seed: 30
Solving Seed: 31
Solving Seed: 32
Solving Seed: 33
Solving Seed: 34
Solving Seed: 35
Solving Seed: 36
Solving Seed: 37
Solving Seed: 38
Solving Seed: 39
Solving Seed: 40
Solving Seed: 41
Solving Seed: 42
Solving Seed: 43
Solving Seed: 44
Solving Seed: 45
Solving Seed: 46
Solving Seed: 47
Solving Seed: 48
Solving Seed: 49
Solving Seed: 50
Solving Seed: 51
Solving Seed: 52
Solving Seed: 53
Solving Seed: 54


"output_exact_pretrain_I=30.csv"